In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

from os import listdir
from os.path import isfile, join
from modules import tratar_nome, return_downloaded, enable_download
import time
import pandas as pd
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

In [2]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.get("https://access.btgpactualdigital.com/login/externo")

In [3]:
conexao = Connect.connect_techdb()
comissao = Connect.import_table(conexao, "comissoes_compilado")
base = Connect.import_table(conexao, "base_btg")
pl_base = Connect.import_table(conexao, "PL Base")
conexao.close()

In [ ]:
def abrir_pagina_performance():
    driver.find_element(By.XPATH, "//img[@class='menu-first-option-title-icon']").click()
    time.sleep(1)
    driver.find_element(By.XPATH, "//a[text()[contains(., 'Plataforma de Performance')]]").click()
    time.sleep(2)

    aba_principal = driver.window_handles[1]
    driver.switch_to.window(aba_principal)

In [ ]:
def expand_shadow_element(element):
    return driver.execute_script("return arguments[0].shadowRoot", element)


def acessar_shadow_root(driver_scrap):
    # Localize o host do shadow-root
    shadow_host = driver_scrap.find_element(By.XPATH, "//kp-composer-editor[@class='full-width']")

    # Expanda o shadow-root
    shadow_root = expand_shadow_element(shadow_host)
    return shadow_root

In [ ]:
def acessar_elemento(sr_scrap):
    acessar_elementos = sr_scrap.find_element(By.CSS_SELECTOR, '.q-btn--unelevated > span:nth-child(2) > span:nth-child(1)')
    acessar_elementos.click()

In [ ]:
def inserir_conta(conta: str):
    try:
        input_conta = driver.find_element(By.XPATH, "//input[@aria-label='Conta do cliente']")
        input_conta.click()
        input_conta.send_keys(Keys.CONTROL + "a")
        input_conta.send_keys(conta)
    except:
        time.sleep(20)
        input_conta = driver.find_element(By.XPATH, "//input[@aria-label='Conta do cliente']")
        input_conta.click()
        input_conta.send_keys(Keys.CONTROL + "a")
        input_conta.send_keys(conta)

In [ ]:
def ultimo_dia(driver_scrap):
    time.sleep(10)
    data_fim = driver_scrap.find_element(By.XPATH, "(//div[@class='q-field__append q-field__marginal row no-wrap items-center'])[2]")
    data_fim.click()
    time.sleep(1)
    voltar_mes = driver_scrap.find_element(By.XPATH, "(//i[text()[contains(.,'chevron_left')]])[1]")
    voltar_mes.click()
    time.sleep(1)
    ultimo_dia = driver_scrap.find_element(By.XPATH, "//div[@class='q-date__calendar-item q-date__calendar-item--in'][last()]")
    ultimo_dia.click()
    time.sleep(1)
    close = driver_scrap.find_element(By.XPATH, "//span[text()[contains(.,'Close')]]")
    close.click()

In [ ]:
def carregar_relatorio(driver_scrap):
    carregar_rel = driver_scrap.find_element(By.XPATH, "//span[text()[contains(.,'Carregar performance')]]")
    carregar_rel.click()

In [ ]:
def gerar_relatorio(sr_scrap, driver_scrap):
    gerar = sr_scrap.find_element(By.CSS_SELECTOR, "span.no-wrap")
    gerar.click()
    time.sleep(1)
    retrato = driver_scrap.find_element(By.XPATH, "(//div[text()[contains(.,'Retrato')]])[2]")
    retrato.click()

In [ ]:
import os
from datetime import datetime

data = datetime.today().strftime("%d-%m-%Y")
diretorio_base = "C:\\Scripts\\relatórios\\"
horario = os.listdir("{}{}".format(diretorio_base, data))[0]

base = pd.read_excel(diretorio_base+"%s\\%s\\base_btg.xlsx" % (data, horario), header=2, dtype={"Conta": str})
base['Assessor'] = [nome.upper() for nome in base['Assessor']]

In [ ]:
baixar = base[
    (base['Assessor'].str.contains("FERNANDO"))
    ]
    
baixar = baixar[(baixar['PL Total'].isnull() == False) &
                (baixar['PL Total'] > 0)]

In [ ]:
baixados = return_downloaded("FERNANDO")
baixar = baixar[~baixar['Conta'].isin([str(x) for x in baixados])]

In [ ]:
len(baixar)

86

In [ ]:
abrir_pagina_performance()

In [ ]:
aba_principal = driver.window_handles[1]
driver.switch_to.window(aba_principal)
shadow_root_driver = acessar_shadow_root(driver)

In [ ]:
baixar = baixar[1:]

In [ ]:
for conta in baixar['Conta']:
    aba_principal = driver.window_handles[1]
    driver.switch_to.window(aba_principal)
    acessar_elemento(shadow_root_driver)
    time.sleep(10)
    try:
        inserir_conta(conta)
    except:
        acessar_elemento(shadow_root_driver)
        inserir_conta(conta)
    time.sleep(3)
    ultimo_dia(driver)
    carregar_relatorio(driver)
    time.sleep(4)
    gerar_relatorio(shadow_root_driver, driver)
    
    while len(driver.window_handles) <= 2:
        time.sleep(0.5)
        
    aba_principal = driver.window_handles[2]
    driver.switch_to.window(aba_principal)
    driver.close()